In [1]:
#Building a NLP NMT English to Spanish using RNN technique as well as Attention mechanism

In [2]:
import tensorflow as tf
from pathlib import Path


url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng")/"spa.txt").read_text()



In [3]:
#Now we shall remove some specific tokens present only in spanish language like inverted exclamation marks, etc
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [4]:
#Creating Textvectorization layets, one per language
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [5]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [6]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [7]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [8]:
tf.random.set_seed(42)
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [9]:
#Encoding the sentences and embedding layer for each language
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [10]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [11]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [12]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [13]:
output_layer

In [14]:
#Lets create a model now
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 123s 34ms/step - loss: 3.2351 - accuracy: 0.3757 - val_loss: 2.6319 - val_accuracy: 0.4458
Epoch 2/10
3125/3125 [==============================] - 81s 26ms/step - loss: 2.2728 - accuracy: 0.4981 - val_loss: 2.0340 - val_accuracy: 0.5369
Epoch 3/10
3125/3125 [==============================] - 75s 24ms/step - loss: 1.7818 - accuracy: 0.5789 - val_loss: 1.6879 - val_accuracy: 0.6013
Epoch 4/10
3125/3125 [==============================] - 75s 24ms/step - loss: 1.4600 - accuracy: 0.6403 - val_loss: 1.4912 - val_accuracy: 0.6390
Epoch 5/10
3125/3125 [==============================] - 73s 23ms/step - loss: 1.2456 - accuracy: 0.6837 - val_loss: 1.3878 - val_accuracy: 0.6600
Epoch 6/10
3125/3125 [==============================] - 75s 24ms/step - loss: 1.0892 - accuracy: 0.7156 - val_loss: 1.3330 - val_accuracy: 0.6718
Epoch 7/10
3125/3125 [==============================] - 81s 26ms/step - loss: 0.9619 - accuracy: 0.7430 - val_loss: 1.3180 

In [15]:
#Now we can not use model.predict here as the decoder also expects inputs from previous predictions
#Now we shall make a fucntion to feed the decoder as well

def translate(sentence_en):
  translation = ""
  for word_idx in range(max_length):
    X = np.array([sentence_en]) #encoder input
    X_dec = np.array(["startofseq " + translation])
    y_proba = model.predict((X, X_dec))[0, word_idx]
    predicted_word_id = np.argmax(y_proba)
    predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
    if predicted_word == "endofseq":
      break
    translation += " " + predicted_word
  return translation.strip()


In [22]:
translate("This is love.")

1/1 [==============================] - 0s 33ms/step


'esto es amor'